<a href="https://colab.research.google.com/github/ApurbaPaul-NLP/Machine-Learning/blob/main/22_08_2022_Introduction_to_modules%2C_layers%2C_and_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A model is, abstractly A function that computes something on tensors.

Here in a Model Some variables can be updated in response to training.

In [1]:
import tensorflow as tf
import numpy as np
# Load the TensorBoard notebook extension
%load_ext tensorboard

In TensorFlow, most high-level implementations of layers and models, such as Keras or Sonnet, are built on the same foundational class: tf.Module.

In [4]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0)).numpy()

30.0

In [8]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


# **What is a "callable"?**

In [9]:
class Foo:
  def __call__(self):
    print ('called')

foo_instance = Foo()
foo_instance()

called


    Consider this foo() as foo.__call__() in the above example.

    Where foo can be any object that responds to __call__. 

When I say any object, I mean it: built-in types, your own classes and their instances.

In the case of built-in types, when you write:

      int('10')

      unicode(10)

You're essentially doing:

        int.__call__('10')

        unicode.__call__(10)

You're essentially doing 

      type(int).__call__(int, '10') and 

      type(unicode).__call__(unicode, '10')



    __call__ makes any object be callable as a function.

    In Python a callable is an object which type has a __call__ method.

To check function or method of class is callable or not that means we can call that function.
    



In [ ]:
class Adder(object):
  def __init__(self, val):
    self.val = val

  def __call__(self, val):
    return self.val + val

func = Adder(5)
print (func(3))

8


In [ ]:
class A:
    def __init__(self,val):
        self.val = val
    def bar(self):
        print ("bar")

obj = A(8)      
callable(obj.bar)

True

# **Here is an example of a two-layer linear layer model made out of modules.**

In [10]:
#First a dense (linear) layer:
class Dense(tf.Module):
  def __init__(self,in_features,out_features,name=None):
    super().__init__(name=name)
    self.w=tf.Variable(tf.random.normal([in_features,out_features]),name='w')
    self.b=tf.Variable(tf.zeros([out_features]),name='b')

  def __call__(self,x):
    y=tf.matmul(x,self.w)+self.b
    return tf.nn.relu(y)  

In [11]:
# The complete model, which makes two layer instances and applies them:
class SimpleModule(tf.Module):
  def __init__(self,name=None):
    super().__init__(name=name)

    self.dense1=Dense(in_features=3,out_features=3)  
    self.dense2=Dense(in_features=3,out_features=2)

  def __call__(self,x):
    x=self.dense1(x)
    return self.dense2(x)  

mymodule=SimpleModule(name="the_Model") 
print(mymodule(tf.constant([[2.,2.,2.]])))   

tf.Tensor([[0.       7.978698]], shape=(1, 2), dtype=float32)


In [12]:
print("Submodules:", mymodule.submodules)

Submodules: (<__main__.Dense object at 0x7f4ae6214b90>, <__main__.Dense object at 0x7f4ae6210510>)


In [13]:
for var in mymodule.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.1072123 , -0.26831588,  1.5347631 ],
       [ 2.2944863 , -0.41903457,  0.28044468],
       [-0.4710366 ,  0.79274863, -0.27444416]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-1.0740916 ,  1.1413937 ],
       [ 0.21701065, -0.5652781 ],
       [-1.9099013 ,  0.45684454]], dtype=float32)> 



In [15]:
import tensorflow as tf

In [41]:
class FlexibleDenseModule(tf.Module):
  def __init__(self,out_features,name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features
    
  def __call__(self,x):
    if not self.is_built:
      self.w=tf.Variable(tf.random.normal([x.shape[-1],self.out_features]),name='w')
      self.b=tf.Variable(tf.zeros(self.out_features),name='b')
      self.is_built=True
    
    y=tf.matmul(x,self.w)+self.b
    return tf.nn.relu(y)

In [47]:
class MySequentialModule(tf.Module):
  def __init__(self,name=None):
    super().__init__(name=name)

    self.dense1=FlexibleDenseModule(out_features=20)
    self.dense2=FlexibleDenseModule(out_features=15)
    self.dense3=FlexibleDenseModule(out_features=10)
    self.dense4=FlexibleDenseModule(out_features=5)
    self.dense5=FlexibleDenseModule(out_features=2)

  def __call__(self,x):
    x=self.dense1(x)
    x=self.dense2(x)
    x=self.dense3(x)
    x=self.dense4(x)
    return self.dense5(x)
mymodel=MySequentialModule(name='The_Model')
print(mymodel(tf.constant([[2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]]).numpy())      )

tf.Tensor([[186.19736 450.93634]], shape=(1, 2), dtype=float32)


# **Saving weights**

You can save a tf.Module as both a checkpoint and a SavedModel.

Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules):

In [48]:
chkp_path = "/content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022"
checkpoint=tf.train.Checkpoint(model=mymodel)
checkpoint.write(chkp_path)

'/content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022'

Checkpoints consist of two kinds of files: the data itself and an index file for metadata. 

The index file keeps track of what is actually saved and the numbering of checkpoints

The checkpoint data contains the variable values and their attribute lookup paths.

In [49]:
!ls /content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022*

/content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022.data-00000-of-00001
/content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022.index


In [50]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense1/b/.ATTRIBUTES/VARIABLE_VALUE', [20]),
 ('model/dense1/w/.ATTRIBUTES/VARIABLE_VALUE', [20, 20]),
 ('model/dense2/b/.ATTRIBUTES/VARIABLE_VALUE', [15]),
 ('model/dense2/w/.ATTRIBUTES/VARIABLE_VALUE', [20, 15]),
 ('model/dense3/b/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('model/dense3/w/.ATTRIBUTES/VARIABLE_VALUE', [15, 10]),
 ('model/dense4/b/.ATTRIBUTES/VARIABLE_VALUE', [5]),
 ('model/dense4/w/.ATTRIBUTES/VARIABLE_VALUE', [10, 5]),
 ('model/dense5/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense5/w/.ATTRIBUTES/VARIABLE_VALUE', [5, 2])]

In [53]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("/content/drive/MyDrive/TENSORLFOW/my_checkpoint-26.08.2022")

# Should be the same result as above
new_model(tf.constant([[2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[186.19736, 450.93634]], dtype=float32)>